# Introduction

## GAMS

GAMS is a high level modeling system for mathematical programming and optimization. It consists of a language compiler and a range of associated solvers. GAMS is designed for modeling and solving linear, nonlinear, and mixed-integer optimization problems. The first presentation of GAMS was in 1976, at the International Symposium on Mathematical Programming (ISMP). It's the first algebraic modeling language (AML). GAMS is a non-free software, it has a proprietary licence.

## Python and Pyomo

### Python

Python is an interpreted, object-oriented, high-level programming language with dynamic semantics. Python supports modules and packages, which encourages program modularity and code reuse. It is developed under an OSI-approved open source license, making it freely usable and distributable, even for commercial use. The first version of Python was published in 1991 by Guido van Rossum, a Dutch programmer.

### Pyomo

Pyomo is a Python-based, open-source optimization modeling language with a diverse set of optimization capabilities for formulating, solving, and analyzing optimization models. Pyomo supports a wide range of problem types, including Linear programming, Nonlinear programming, Mixed-integer linear programming and more

### Why Pyomo ?

GAMS is a software which isn't free, then students and people who don't pay for it can't use the model as much as they want. So this is why the CIRED wanted a translation in an open-source language. Python with Pyomo offers a very good alternative of GAMS, It is easy-to-use, and more and more school uses Python to teach algorithm.

## The Translation

### Presentation of the work

### Things to know before reading

There is some things to know before programming in *Python* : 
- Variables are created when you assign a value to it. Python has no command for declaring a variable. The type of the variable depends of the value assigned to it. 
- There is no `;` at the end of an instruction, in *Python*, the end of an instruction is the end of the line.
    - But for a better reading of, we can use `\` to put the end of an instruction on a new line.
- The identation of your code is very important. Python uses indentation to indicate a block of code. If you skip the identation, Python will return an error.
- A tuple is an unchangeable collection of elements.  It can be any types. Tuple are define using `()`.

# Pyomo Formulation

## Imports

In Python, there is a lot of libraries and modules created by the community like pyomo. We can import them with ``import``
For this program we will use : 
* Pandas, a powerful, easy to use and open source data analysis and data manipulation tool.
* Csv, A tool for python to write into csv files.
* Time, A tool which will help us to measure the lenght of the program.
* Sys, Which will help us to manage error in the program.

On the first line, the word **as** is used to set an alias. Each time we want to call a *pyomo.environ* function, we just have to call *pyo* instead.

In [1]:
#Import Modules
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
import pandas as pd
import csv
import time
import sys

## Creation of the model

In pyomo, the model has to be declared first. It will be created as an object. \
Each component we will define will be an attribute of this object. \
There are two type of models in Pyomo, `ConcreteModel` and `AbstractModel`.\
Eoles is a `ConcreteModel`.

The suffix component of Pyomo is here to set a dual variable. Used to get the marginal value of an equation.

In [2]:
#Creation of the model
model = pyo.ConcreteModel()

#Dual Variable, used to get the marginal value of an equation.
model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

## Inputs

The method ``read_csv`` of *Pandas* is used to read a csv file into a DataFrame :
* The first argument is the ``PATH`` of the file.
* ``index_col``   : Column to use as the index of the DataFrame.
* ``squeeze``     : Boolean which is true in our program then if the parsed data only contains one column it will return a series.
* ``header``      : Specify if the csv file contains a header.


In [3]:
# Production profiles of VRE
load_factor = pd.read_csv("inputs/vre_profiles2006new.csv", index_col=[0, 1], squeeze=True, header=None)
# maximum capacities of the technologies in GW
capa_max = pd.read_csv("inputs/max_capas_elec_new.csv", index_col=0, squeeze=True, header=None)
# existing capacities of the technologies by December 2017 in GW
capa_ex = pd.read_csv("inputs/existing_capas_elec_new.csv", index_col=0, squeeze=True, header=None)

##  Equivalent in GAMS  ##
#   parameter load_factor(vre,h) 'Production profiles of VRE'
#   /
#   $ondelim
#   $include  inputs/vre_profiles2006new.csv
#   $offdelim
#   /;
#   parameter capa_max(vre) 'maximum capacities of the technologies in GW'
#   /
#   $ondelim
#   $include  inputs/max_capas_elec_new.csv
#   $offdelim
#   / ;

## Sets

Sets are created with the ``Set`` attributs of *Pyomo*. We are passing elements in a table with the option _initialize_ : \
`model.set_name = pyo.Set(initialize=["elem_1","elem_2","..."])` \
There is also `RangeSet` which needs the first and the last elements of the set : \
`model.set_range_name = pyo.RangeSet(0,10)`

There is no need to define the domain of a subset.

In [4]:
#Declaration of the Sets
model.tec = pyo.Set(initialize=["offshore_f", "river", "lake", "ccgt"])
model.vre = pyo.Set(initialize=["offshore_f", "river"])
#Declaration of a RangeSet from 0 to 8759.
model.h = pyo.RangeSet(0,8759)

##  Equivalent in GAMS   ##
#   sets     h          'hour in the model'     /0*8759/
#            tec        'technology'            /offshore_f, river, lake, ccgt /
#            vre(tec)   'variable tecs'         /offshore_f, river /
#   ;

## Initial Values and Bounds

For the variables of the model, we first declared a function which will return the lower bound and the upper bound of the variable in a tuple. Arguments of the function are the model object (here "model") and the parameters of the variable. \
The variable will iterate on this function through its parameters so we can defined for each parameters, bounds we want : 
```
def function_name(model_object,argument):
    if argument in [condition] :
        return (lower_value,upper_value)
    else :
        return (value_lower,value_upper)
```

In [5]:
def capa_bounds(model,i):
    if i == 'phs' :
        #In the pyomo model, data are in miscellaneous.csv file
        return (miscellaneous['phs_discharging_lower'],miscellaneous['phs_discharging_upper'])
    elif i in model.vre:
        return(None,capa_max[i])
    else :
        return(None,None)

##  Equivalent in GAMS  ##
#   CAPA.lo('phs') = 4.94;
#   CAPA.up('phs') = 9.3;
#   CAPA.up(vre) = capa_max(vre);

## Variables

Variables are created with the ``Variable`` attributs of *Pyomo*. We are passing sets as an argument with some options :
```
model.var_name = pyo.Var(
    (sets_names),
    within=domain,
    initialization=initial_value,
    bounds=function we defined earlier
    )
```
If there is many sets in the parameters of a variable, you have to declare a tuple and set this tuple with a for loop. You can see how ot works in the example  : **model.gene** \
The option ``within=pyo.NonNegativeReals``, is from *Pyomo*, it means all positive reals including zero.\
You can find more domain set by *Pyomo* in the documentation.


In [6]:
    # Hourly energy generation in GWh/h
model.gene = \
    pyo.Var(((tec, h) for tec in model.tec for h in model.h), within=pyo.NonNegativeReals,initialize=0)
    # Overall yearly installed capacity in GW
model.capa = \
    pyo.Var(model.tec, within=pyo.NonNegativeReals, initialize=0,bounds=capa_bounds)

##  Equivalent in GAMS  ##
#   variables   GENE(tec,h) 'hourly energy generation in TWh'
#               CAPA(tec)   'overall installed capacity in GW'
#
#   positive variables GENE(tec,h),CAPA(tec);


## Fixed Values

Fixed values are set with the variable's function `fix()`. We just have to call this function with the value we want between parenthesis : 
```
model.variable_name['element'].fix(value)
```

In [7]:
model.capa['lake'].fix(capa_ex['lake'])
model.capa['river'].fix(capa_ex['river'])

##  Equivalent in GAMS  ##
#   CAPA.fx('river')= capa_ex('river');
#   CAPA.fx('lake') = capa_ex('lake');

## Constraints
 
With *Pyomo*, constraints creation needs two steps. First we will defined a function which will return the equation of the constraint. Then we will create the constraint with the ``Constraint`` attribut of *Pyomo*. This `Constraint` will have the function we just defined into a ``rule`` argument.
The order of the argument in the definition of the function is very important, it has to be the same as in the constraint creation despite the constraint doesn't need the model object as a parameters :
```
def contraint_name_rule(model_object,argument):
    return model_object.variable_name[argument] == model_object.name_variable[argument]

model.constraint_name = pyo.Constraint(argument,rule=constraint_name_rule)
```

Relational Operators are differents between GAMS and Python :

GAMS    | Python    | Definition                |
---     | ---       | ---                       |
=e=     | ==        | Equal to                  |
=g=     | >=        | Greater than or equal to  |
=l=     | <=        | Less than or equal to     |

In [8]:
def generation_vre_constraint_rule(model, h, vre):
    return model.gene[vre, h] == model.capa[vre] * load_factor[vre,h]

model.generation_vre_constraint = \
    pyo.Constraint(model.h, model.vre, rule=generation_vre_constraint_rule)

##  Equivalent in GAMS ##
#   equation    gene_vre    'vre profiles generation";
#
#   gene_vre(vre,h)..   GENE(vre,h) =e= CAPA(vre)*load_factor(vre,h);

The definition of the objective is a little bit different between GAMS and Pyomo. In GAMS, the objective is a constraint like an other. \
In Pyomo, the objective rule function doesn't return an equation but directly the value of the objective. But the creation of the `Objective` is pretty similar as the creation of a `Constraint`. We just have to use `Pyo.Objective` instead as `Pyo.Constraint`

In [9]:
#This objective function is not accurate, it's juste here to illustrate the concept
def objective_rule(model):
    return (sum(model.capa[tec] for tec in model.tec))/1000

model.objective = pyo.Objective(rule=objective_rule)

##  Equivalent in GAMS  ##
#   equation    obj     'the final objective";
#
#   obj..       COST =e= sum(tec,CAPA(tec))/1000;  

## Solve Statement

To Solve the model, we have to create the solver with `SolverFactory`. Then we can solve it with `solve(model_object)`

In [10]:
opt = SolverFactory('gurobi')
results = opt.solve(model)

##  Equivalent in GAMS  ##
#   model Eoles_elec /all/;
#   option lp = gurobi;

## Outputs

Outputs are managed with a library : **csv**. \
There is an try/except instruction to manage errors : the program will try to write in the file and if this doesn't work, an error will be return. With this instruction, the programm will not stop if there is an error about an output. \
`with` allows us to open a file (to write in it) without closing it after. The file will be closed at the end of the instruction.  \
To access a value from a Pyomo attribut (Constraint or Variable) : ``pyo.value(model.attribut_name)``.

This is like `display` in GAMS.

In [11]:
"""Creation of an output for eoles :"""

#Name of the model, only used to name outputs
model_name = "Example"

#Name of the file
out_file = "outputs/eoles_" + model_name + "_summary.csv"

#Try / Except : To manage errors
try:
    #As il like "alias", here we open the file and we set an alias for this file.
    with open(out_file,"w",newline="") as output:

        #Definition of a writer of the csv library.
        output_writer = csv.writer(output)

        #We set the value we want in a table then we write it as a row.
        data = ["Cost",pyo.value(model.objective),"..."]
        output_writer.writerow(data)

#This instruction we be execute only if the program catch an error.
except Exception as e:
    print("There is an Error :",e.args)

# References

* __Eoles__ : http://www.centre-cired.fr/quel-mix-electrique-optimal-en-france-en-2050/
* __Python__ : https://docs.python.org/3/
* __Pyomo__ : https://pyomo.readthedocs.io/en/stable/index.html
* __GAMS__ : https://www.gams.com/latest/docs/
* __Pandas__ : https://pandas.pydata.org/pandas-docs/stable/index.html

If you have any question about this document or about the program you can ask on : *nilam.deoliveiragill@gmail.com*

